In [14]:
import cv2
import mediapipe as mp
import csv
import os
import time

# 데이터 위치 셋업
output_dir = r"data\continuous"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 파일 생성
timestamp_str = time.strftime("%Y%m%d_%H%M%S")

# 이름 설정. (ex. 동작 번호 이름 매겨주세요 123 345 567)
csv_filename = os.path.join(output_dir, f"12345.csv")

# 미디어파이프 기본 설정값.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,         # For video stream
    max_num_hands=2,                 # Maximum 2 hands
    min_detection_confidence=0.5,    # Detection confidence threshold
    min_tracking_confidence=0.5      # Tracking confidence threshold
)

mp_drawing = mp.solutions.drawing_utils

# 웹캠 오픈
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

exit_requested = False
print("Continuous data collection started.")


with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    
    
    header = ["timestamp", "hand_detected"]
    for hand in ['hand1', 'hand2']:
        for i in range(21):
            header.extend([f"{hand}_landmark{i}_x", f"{hand}_landmark{i}_y", f"{hand}_landmark{i}_z"])
    writer.writerow(header)

    while not exit_requested:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not retrieve frame.")
            break

        
        frame = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)
        image_rgb.flags.writeable = True

        
        ts = time.time()
        row = [ts]

        
        if results.multi_hand_landmarks:
            hand_detected = 1
        else:
            hand_detected = 0
        row.append(hand_detected)

        if results.multi_hand_landmarks:
           
            landmarks_all = []
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                landmarks_all.append(landmarks)
            # 첫 번째 손 정보 저장
            row.extend(landmarks_all[0])
            # 두 번째 손 정보: 만약 검출되지 않았다면 63개의 0으로 채움
            if len(landmarks_all) >= 2:
                row.extend(landmarks_all[1])
            else:
                row.extend([0] * 63)
        else:
            # 손이 전혀 검출되지 않은 경우, 두 손 모두 126개의 0으로 채움
            row.extend([0] * 126)

        writer.writerow(row)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Continuous Data Collection", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            exit_requested = True

# 끝
cap.release()
cv2.destroyAllWindows()
hands.close()
print(f"Exiting. Data saved to {csv_filename}")

Continuous data collection started.
Exiting. Data saved to data\continuous\continuous_20250218_163302.csv


In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0
